# Intro to SQL and Python

_Note: key terms will be bolded throughout this document._

You're currently running a Jupyter notebook! 

This is an interactive coding environment that allows you to run Python code in **cells** as seen below.

Try executing the cell below by clicking into it and pressing `SHIFT + ENTER`. You should see the text to the left of the cell, `In []`, update to `In [1]`. This number represents the order in which you've executed cells in your notebook.

In [2]:
# In Python, comments start with a hashtag at the beginning of a line
# This cell imports the psycopg2 library which we previously installed
# Importing a library allows us to use it during our current session
import psycopg2

We've successfully **imported** our library! For your reference, the `psycopg2` [library](https://www.psycopg.org/docs/) is an **open-source package** which allows us to interact with a live SQL database in our Python environment. Below, we're going to use this package to work with a popular open-source version of SQL called [Postgres](https://en.wikipedia.org/wiki/PostgreSQL).

Before we get any further, if you're looking to get more comfortable with the syntax of Python, I'd recommend perusing the following interactive tutorials:
- [Printing Hello, World!](https://www.learnpython.org/en/Hello%2C_World%21)
- [Variables and Types](https://www.learnpython.org/en/Variables_and_Types)
- [Lists](https://www.learnpython.org/en/Lists)
- [Conditions](https://www.learnpython.org/en/Conditions)
- [Loops](https://www.learnpython.org/en/Loops)
- [Functions](https://www.learnpython.org/en/Loops)

I'll do my best to explain various CS concepts in simple terms as we move along. The above tutorials should be considered _optional_ learning if you want to get better at working with Python syntax. Other optional content is formatted in italics below.

Now — click into the cell below and hit `SHIFT + ENTER`. You should see the cell count increment to `In [2]`.

In [3]:
connection = psycopg2.connect(
    database="", # database name
    user="", # database user's username
    password="", # database user's password
    host="hbs.cm4xib1x9qhd.us-east-1.rds.amazonaws.com", # web address
    port='5432' # unique connection identifier 
)

What's going on in this cell? 

We're calling a **function** that exists in the `psycopg2` package called `connect()`. This function establishes a connection to a database somewhere on the web — in other words, it logs in! 

It accepts various **parameters** such as `database`, `user`, `password`, `host`, and `port`. *(You can optionally read more about ports [here](https://en.wikipedia.org/wiki/Port_(computer_networking)).)*

We're **assigning** the result of this function execution to a **variable** which we have called `connection`. Note that we could call this variable whatever we want, such as `pizza`.

The specific Postgres database that we're working with is one that I've set up on AWS for the purpose of this exercise. Once we've established this connection, we can now query the database!

_If you're curious about how the `connect()` function works, the [documentation](https://www.psycopg.org/docs/connection.html?highlight=connect) for the `psycopg2` library explains this in more detail._

In [4]:
# First, we create a database cursor 
from psycopg2 import extras
cursor = connection.cursor(cursor_factory=psycopg2.extras.DictCursor)

What did we do here? If you said that we:
- Called the `cursor()` function of our `connection`
- Assigned the result of this function to a variable called `cursor`

Then you're correct! 

_We conduct a separate import of the `extras` module from `psycopg2` so that the resulting `cursor` is formatted slightly neater than the default setting. You don't need to understand what's happening with the `cursor_factory` parameter; just know that it makes our life easier._

Again, we could have called our `cursor` variable `ice_cream` if we wanted to. Note that the `connection.cursor()` function is explicitly defined in the documentation as part of the `connection` that is created by `psycopg2`. 

We would not be able to call `connection.ice_cream()` as that isn't a real thing. However, if we had stored the `connection` in a different variable, such as `pizza`, we would be able to call `pizza.cursor()`. 

_See the `psycopg2` [docs](https://www.psycopg.org/docs/cursor.html#cursor) for more details!_

### Querying the database

![Let's get down to business](https://pyxis.nymag.com/v1/imgs/433/898/17ff3bd1a1e20e75702cc4b1a99bc3430e-li-shang.2x.rsocial.w600.jpg)

Now let's down to business!

Just kidding. Before we can query our database, we have to know what it actually looks like!

A database is made up of many individual **tables**, each of which can be analogized to a spreadsheet. The **columns** of a table represent specific attributes, and the **rows** of a table represent individual objects.

Let's construct a SQL query that looks at our database's metadata. This will get us a list of all the tables in our database.

So ... what **clauses** make up a SQL query? Queries that involve **reading data** (often what a PM is doing) have the following components:
1. `SELECT column1, column2, ...`
2. `FROM table_name`
3. `WHERE condition;`

**Don't forget the semicolon!**

Given this structure, we can construct a query that retrieves all of the tables in our database:
```
SELECT table_name
FROM information_schema.tables
WHERE table_schema='public';
```

How did we know to construct our query like this? The [Postgres documentation](https://www.postgresql.org/docs/current/infoschema-tables.html) outlines all of this information about how metadata tables are standardized. Yup, the answer is "Read the manual".

_Looking at the docs, we see that there is a standard `table_name` column which exists on the standard `information_schema.tables` metadata table.  However, since Postgres has a lot of private metadata tables, we should also specify that we only want tables where the `table_schema` column is public, as that represents tables which store real data._

Now, how do we execute this SQL query against the database using our `cursor` which we constructed above?

In [5]:
# Call the cursor.execute() function which takes one parameter, our query
# In Python, multi-line text strings go in between triple-quotes
cursor.execute(
    """
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'public';
    """
)

# After executing the query, we MUST also call cursor.fetchall()
# This returns the results of our query, otherwise nothing happens
cursor.fetchall()

[['titles'], ['people'], ['revenues'], ['roles']]

We got our results! Why do they look kind of funky? 

Well, the square brackets in Python represent a `list` data type. We simply received the results of our function call as a nested list of lists. 

Now, how do we figure out what the columns are on each of these tables? 

We perform a similar metadata query on another Postgres metadata table (this time, `information_schema.columns` — see [docs](https://www.postgresql.org/docs/current/infoschema-columns.html)) and we specifically add the condition that `table_name = 'titles'`. Note that the name of the table, being a text string, must be enclosed in quotes.

Let's run this query for each of our tables!

In [6]:
cursor.execute(
    """
    SELECT column_name
    FROM information_schema.columns
    WHERE table_schema = 'public' AND table_name = 'titles';
    """
)
cursor.fetchall()

[['id'],
 ['adaptation'],
 ['domestic_gross'],
 ['franchise'],
 ['genre'],
 ['international_gross'],
 ['mpaa_rating'],
 ['plot'],
 ['poster_url'],
 ['production_budget'],
 ['runtime'],
 ['metascore'],
 ['rt_critics'],
 ['rt_users'],
 ['imdb_score'],
 ['theatrical_release'],
 ['title'],
 ['three_d']]

In [7]:
cursor.execute(
    """
    SELECT column_name
    FROM information_schema.columns
    WHERE table_schema = 'public' AND table_name = 'people';
    """
)
cursor.fetchall()

[['id'], ['name'], ['gender'], ['race'], ['birthday']]

In [8]:
cursor.execute(
    """
    SELECT column_name
    FROM information_schema.columns
    WHERE table_schema = 'public' AND table_name = 'roles';
    """
)
cursor.fetchall()

[['title_id'], ['person_id'], ['role_type']]

In [9]:
cursor.execute(
    """
    SELECT column_name
    FROM information_schema.columns
    WHERE table_schema = 'public' AND table_name = 'revenues';
    """
)
cursor.fetchall()

[['title_id'], ['week_number'], ['gross'], ['theaters']]

Have you guessed what data is contained in our database? It's movies!

We have a table of `titles` corresponding to individual movies and a table of `people` corresponding to actors and directors. 

But why do we have a table of `roles` and `revenues`? This gets at a fundamental question of **data modeling** — where do you store data that pertains to multiple tables? 

Consider that one row in the `titles` table corresponds to one movie, and one row in the `people` database corresponds to one actor. What happens if we store the cast in `titles` and it changes? What happens if we store an actor's filmography in `people` and it changes? What happens if we store the data in both locations? How much work would it be to update the data, and which table contains the source of truth?

Well, because our movies have relationships with actors, we can create a completely separate table that represents the relationship between `titles` and `people`. This third table is the single source of truth! 

Notice that every movie in `titles` has an `id` property. This is a unique value which we call a **primary key**, which allows us to differentiate between different movies. Similarly, different actors in `people` have unique `id` values. When we want to capture that Tom Holland is starring in a new Spider-Man movie, we can use Tom Holland's `id` from `people` and the movie's `id` from `titles` to create a new entry in `roles`. 

As you can see, the `roles` table has `title_id` and `person_id` columns. These are considered **foreign keys** because they correspond to the primary keys of other tables. 

Similarly, we store new weekly revenue numbers in the `revenues` table by associating each entry with a movie's `title_id` foreign key. 

### Some simple examples

In [10]:
# Retrieve all columns for movies with a runtime of over 200 minutes
cursor.execute(
    """
    SELECT *
    FROM titles
    WHERE runtime > 200;
    """
)
cursor.fetchall()

[['aa2b71bf-ac03-4e6a-845b-0f1000026553',
  'false',
  Decimal('145526'),
  None,
  'Biography',
  Decimal('145526'),
  'Unrated',
  "Ilich Ramirez Sanchez, aka 'Carlos,' is a Venezuelan-born Marxist revolutionary who aligns himself with the Palestinian cause and becomes the world's most notorious terrorist. He leads assaults on the meeting of OPEC ministers, taking them hostage and flying them from country to country seeking asylum, one of the most daring acts of terrorism in history. From his earliest days as an apprentice in the revolutionary movement to his subsequent downfall, Carlos becomes a figure of legend.",
  'http://ia.media-imdb.com/images/M/MV5BMTkwNDU4Mzc4OV5BMl5BanBnXkFtZTcwMzEyMTA3NQ@@._V1_SX300.jpg',
  Decimal('18000000'),
  Decimal('333'),
  Decimal('94'),
  Decimal('94'),
  Decimal('85'),
  Decimal('7.7'),
  datetime.datetime(2010, 10, 15, 0, 0),
  'Carlos',
  False],
 ['b3dc2642-03d2-4d50-9b2b-fe215be4429a',
  'novel',
  Decimal('12882934'),
  'Gettysburg',
  'Dram

In [9]:
# Retrieve Tom Holland
cursor.execute(
    """
    SELECT *
    FROM people
    WHERE name = 'Tom Holland';
    """
)
cursor.fetchall()

[['4e7b6c16-0381-422a-85dd-91a2cd7a38d5',
  'Tom Holland',
  'Male',
  'White',
  datetime.datetime(1996, 6, 1, 0, 0)]]

In [10]:
# Retrieve all of Tom Holland's movies
cursor.execute(
    """
    SELECT 
        t.id, 
        t.adaptation,
        t.domestic_gross,
        t.franchise,
        t.genre,
        t.international_gross,
        t.mpaa_rating,
        t.plot,
        t.poster_url,
        t.production_budget,
        t.runtime,
        t.metascore,
        t.rt_critics,
        t.rt_users,
        t.imdb_score,
        t.theatrical_release,
        t.title,
        t.three_d
    FROM titles AS t
    JOIN roles ON roles.title_id = t.id
    WHERE roles.person_id = '4e7b6c16-0381-422a-85dd-91a2cd7a38d5';
    """
)
cursor.fetchall()

[['07428f8a-d0c4-4d43-875f-237d4b281a74',
  'nonfiction',
  Decimal('25020758'),
  None,
  'Action',
  Decimal('25020758'),
  'PG-13',
  "A recounting of a New England whaling ship's sinking by a giant whale in 1820, an experience that later inspired the great novel Moby-Dick.",
  'http://ia.media-imdb.com/images/M/MV5BMjA5NzUwODExM15BMl5BanBnXkFtZTgwNjM0MzE4NjE@._V1_SX300.jpg',
  Decimal('100000000'),
  Decimal('121'),
  Decimal('47'),
  Decimal('42'),
  Decimal('56'),
  Decimal('7'),
  datetime.datetime(2015, 12, 11, 0, 0),
  'In the Heart of the Sea',
  True],
 ['bff03ce2-971c-4ac5-b301-c62b9e30c394',
  'false',
  Decimal('19019882'),
  None,
  'Drama',
  Decimal('19019882'),
  'PG-13',
  'A regular family - Maria (Naomi Watts), Henry (Ewan McGregor) and their three kids - travel to Thailand to spend Christmas. They get an upgrade to a villa on the coastline. After settling in and exchanging gifts, they go to the pool, like so many other tourists. A perfect paradise vacation until a

It's worth going into some detail as to how we performed this last query. 

Recall from above that there are three clauses to a SQL query:
1. `SELECT column1, column2, ...`
2. `FROM table_name`
3. `WHERE condition;`

What happens when we want to query all of Tom Holland's movies? We need to query information from `titles`, `roles`, and `people`! 

First, we found that Tom Holland's `id` in the `people` table is: `'4e7b6c16-0381-422a-85dd-91a2cd7a38d5'`.

Then, we constructed a query that **joined** the `titles` and `roles` tables. Recall that `titles.id` is a foreign key for `roles.title_id`. This means that we can perform a `JOIN` operation that combines the two tables where the values from one column are equal to the values of another column!

We'll add a fourth clause to our SQL query framework:
1. `SELECT column1, column2, ...`
2. `FROM table1`
3. `JOIN table2 ON table1.columnA = table2.columnB`
4. `WHERE condition;`

Finally, notice that we reference the `titles` table in our query as a shorter alias, `t`. We can use this to simplify query writing, especially where typing out a full table name multiple times might be cumbersome. 

If we modify the `FROM` clause to be `FROM table1 as t`, then all references to `table1` in our query must instead use the alias `t` or our query will fail due to malformed syntax. 

Our framework now looks like this:
1. `SELECT t.column1, t.column2, ...`
2. `FROM table1 AS t`
3. `JOIN table2 ON table1.columnA = table2.columnB`
4. `WHERE condition;`

Using this knowledge, let's write another `JOIN` query to select all of the revenue for Brad Pitt's movies.

In [11]:
# Retrieve Brad Pitt
cursor.execute(
    """
    SELECT *
    FROM people
    WHERE name = 'Brad Pitt';
    """
)
cursor.fetchall()

[['43ccd6ee-145e-44f0-b8b1-e06f0b2cf801',
  'Brad Pitt',
  'Male',
  'White',
  datetime.datetime(1963, 12, 18, 0, 0)]]

In [12]:
# Retrieve the week-by-week gross of Brad Pitt's movies
cursor.execute(
    """
    SELECT 
        r.title_id,
        r.week_number,
        r.gross,
        r.theaters,
        t.title
    FROM revenues AS r
    JOIN (
        SELECT t.id, t.title
        FROM titles AS t
        JOIN roles ON roles.title_id = t.id
        WHERE roles.person_id = '43ccd6ee-145e-44f0-b8b1-e06f0b2cf801'
    ) AS t
    ON r.title_id = t.id;
    """
)
cursor.fetchall()

[['14595dfb-6582-41ac-bb91-93b199dce5b0',
  Decimal('4'),
  Decimal('19466'),
  Decimal('32'),
  'By the Sea'],
 ['14595dfb-6582-41ac-bb91-93b199dce5b0',
  Decimal('3'),
  Decimal('125665'),
  Decimal('142'),
  'By the Sea'],
 ['14595dfb-6582-41ac-bb91-93b199dce5b0',
  Decimal('2'),
  Decimal('265732'),
  Decimal('126'),
  'By the Sea'],
 ['14595dfb-6582-41ac-bb91-93b199dce5b0',
  Decimal('1'),
  Decimal('127597'),
  Decimal('10'),
  'By the Sea'],
 ['8ab470a7-aebf-43c8-8c88-82536c54cc52',
  Decimal('17'),
  Decimal('82252'),
  Decimal('108'),
  'The Big Short'],
 ['8ab470a7-aebf-43c8-8c88-82536c54cc52',
  Decimal('16'),
  Decimal('69852'),
  Decimal('67'),
  'The Big Short'],
 ['8ab470a7-aebf-43c8-8c88-82536c54cc52',
  Decimal('15'),
  Decimal('188582'),
  Decimal('133'),
  'The Big Short'],
 ['8ab470a7-aebf-43c8-8c88-82536c54cc52',
  Decimal('14'),
  Decimal('397323'),
  Decimal('222'),
  'The Big Short'],
 ['8ab470a7-aebf-43c8-8c88-82536c54cc52',
  Decimal('13'),
  Decimal('672744')

What the heck is going on up there? Well, it turns out that you can `JOIN` not just tables, but also other queries! 

So, we took our query from the previous example, which gives us the list of Brad Pitt's movies, and we joined it with the `revenues` table to get the week-by-week revenues of all of those movies. 

Notice the same structure for applying a `JOIN` and for aliasing tables with `AS`! 

### One more thing...

![One more thing](https://i0.wp.com/9to5mac.com/wp-content/uploads/sites/6/2021/03/One-More-Thing-trademark.jpg?resize=1200%2C628&quality=82&strip=all&ssl=1)

There's a library that comes pre-installed with Python Anaconda called `pandas` which can give you superpowers when it comes to performing data analysis. 

Let's take a look at how we can use `pandas` and our knowledge of SQL to really do some cool stuff.

In [15]:
# Import the package — notice that we can also alias here!
import pandas as pd

In [16]:
# Let's take the SQL from above and assign it to a variable
SQL_FOR_PANDAS = """
    SELECT 
        r.title_id,
        r.week_number,
        r.gross,
        r.theaters,
        t.title
    FROM revenues AS r
    JOIN (
        SELECT t.id, t.title
        FROM titles AS t
        JOIN roles ON roles.title_id = t.id
        WHERE roles.person_id = '43ccd6ee-145e-44f0-b8b1-e06f0b2cf801'
    ) AS t
    ON r.title_id = t.id;
"""

It turns out that `pandas` has a function called `pandas.read_sql` which takes two parameters, the SQL query itself and a connection to a database. What happens when we pass that in?

Try running the below cell.

In [17]:
dataframe = pd.read_sql(SQL_FOR_PANDAS, connection)
dataframe

,title_id,week_number,gross,theaters,title
0,14595dfb-6582-41ac-bb91-93b199dce5b0,4.0,19466.0,32.0,By the Sea
1,14595dfb-6582-41ac-bb91-93b199dce5b0,3.0,125665.0,142.0,By the Sea
2,14595dfb-6582-41ac-bb91-93b199dce5b0,2.0,265732.0,126.0,By the Sea
3,14595dfb-6582-41ac-bb91-93b199dce5b0,1.0,127597.0,10.0,By the Sea
4,8ab470a7-aebf-43c8-8c88-82536c54cc52,17.0,82252.0,108.0,The Big Short
...,...,...,...,...,...
508,4deebcb0-b0fd-4ddd-8feb-9605b05cfe95,5.0,332183.0,157.0,Allied
509,4deebcb0-b0fd-4ddd-8feb-9605b05cfe95,4.0,1898105.0,414.0,Allied
510,4deebcb0-b0fd-4ddd-8feb-9605b05cfe95,3.0,5575465.0,3018.0,Allied
511,4deebcb0-b0fd-4ddd-8feb-9605b05cfe95,2.0,9756020.0,3160.0,Allied


Look at that! `pandas` gives us a tabular view of the data that we just queried from a SQL database hosted on AWS!

But can we do anything with this data? 

I want to convince you that `pandas` is trivially capable of performing powerful analysis that oftentimes is difficult or even impossible to script in Excel.

Consider the question: 

**"For each of Brad Pitt's limited release films (premiering in less than 100 theaters in its first week), how many eventually went on to achieve a peak weekly box office of over $10 million?"**

In [17]:
limited_releases = dataframe[(dataframe['week_number'] == 1.0) & (dataframe['theaters'] < 100)]['title']
for movie in limited_releases:
    weekly_revenues = dataframe[dataframe['title'] == movie]
    peak = weekly_revenues['gross'].idxmax()
    r = weekly_revenues.loc[[peak]].values[0]
    if r[2] > 10_000_000:
        print(f'{r[4]} peaked in week {r[1]} with ${r[2]} in {r[3]} theaters.')

The Big Short peaked in week 3.0 with $18503730.0 in 1585.0 theaters.
Snatch peaked in week 4.0 with $10946720.0 in 1444.0 theaters.
Legends of the Fall peaked in week 4.0 with $16714827.0 in 2005.0 theaters.
Twelve Monkeys (12 Monkeys) peaked in week 2.0 with $18798865.0 in 1533.0 theaters.


Just ... Se7en(!) lines of code and we have our answer.

For more complex analysis, we'll have to brush up on our Python skills, and we'll have to dive deeper into the [documentation](https://pandas.pydata.org/docs/) for `pandas`. 

But that's for another day.